In [56]:
import pandas as pd

# Definición de las clases Pedido, Camion, Puerta, y CEDIS
class Camion:
    def __init__(self, id_camion, contenido , fecha_salida, origen):
        # id_camion
        self.id_camion = id_camion
        # id_orden
        self.contenido = contenido
        # fecha m/d/a
        self.fecha = fecha_salida
        # origen
        self.origen = origen
        # estado del camion "esperando" | "descargando" | "retirado"
        self.estado = "espera"
        # tiempo de descarga None. Inicializa en cuanto sea su turno de descargar, finaliza al salir.
        self.tiempo_descarga = None

class Orden:
    def __init__(self, id_orden, orden_cliente, tipo_de_orden, productos):
        self.id_orden = id_orden
        self.orden_cliente = orden_cliente
        # tipo de orden fp -> cliente grande, pk -> cliente chico
        self.tipo_de_orden = tipo_de_orden
        # productos {producto: [cantidad original, cantidad solicitada, cantidad asignada, cantidad empaquetada]}
        self.productos = productos
        self.estado = self.verificar_estado()

    def verificar_estado(self):
        # Verificar si todas las cantidades asignadas y empaquetadas son iguales a las solicitadas
        for cantidades in self.productos.values():
            cantidad_solicitada = cantidades[1]
            cantidad_asignada = cantidades[2]
            cantidad_empaquetada = cantidades[3]
            if cantidad_asignada < cantidad_solicitada or cantidad_empaquetada < cantidad_solicitada:
                self.estado = "pendiente"
                return
        self.estado = "completa"

class Puerta:
    def __init__(self, id_puerta):
        self.id_puerta = id_puerta
        self.estado = 'libre'
        self.camion_actual = None

    def asignar_camion(self, camion):
        if self.estado == 'libre':
            self.camion_actual = camion
            self.estado = 'ocupada'
            camion.estado = 'descargando'
            print(f"Puerta {self.id_puerta} asignada a Camión {camion.id_camion}")

    def liberar_puerta(self):
        if self.estado == 'ocupada':
            print(f"Puerta {self.id_puerta} liberada de Camión {self.camion_actual.id_camion}")
            self.camion_actual.estado = 'retirado'
            self.camion_actual = None
            self.estado = 'libre'

class Patio:
    def __init__(self):
        # Lista de camiones esperando en el patio
        self.camiones_en_espera = []
    
    def agregar_camion(self, camion):
        """
        Agrega un camión a la lista de espera del patio.
        """
        self.camiones_en_espera.append(camion)
        camion.estado = "esperando"
        print(f"Camión {camion.id_camion} agregado a la zona de espera.")
    
    def remover_camion(self, camion):
        """
        Remueve un camión de la lista de espera del patio.
        """
        if camion in self.camiones_en_espera:
            self.camiones_en_espera.remove(camion)
            print(f"Camión {camion.id_camion} removido de la zona de espera.")
    
    def mostrar_camiones_en_espera(self):
        """
        Muestra los camiones actualmente en espera en el patio.
        """
        print("Camiones en espera en el patio:")
        for camion in self.camiones_en_espera:
            print(f"- Camión {camion.id_camion}:  Estado: {camion.estado}, Contenido {camion.contenido}")
    
    def asignar_camion_a_puerta(self, cedis):
        """
        Asigna el primer camión en la lista de espera a una puerta disponible en el CEDIS.
        """
        if not self.camiones_en_espera:
            print("No hay camiones en espera.")
            return

        # Buscar una puerta libre en el CEDIS
        for puerta in cedis.puertas:
            if puerta.estado == 'libre':
                # Asignar el primer camión en espera a la puerta
                camion = self.camiones_en_espera.pop(0)
                puerta.asignar_camion(camion)
                print(f"Camión {camion.id_camion} asignado a la {puerta.id_puerta}.")
                return

        print("No hay puertas disponibles en el CEDIS. Camión en espera.")

class CEDIS:
    def __init__(self, n_puertas, path_inventario):
        # Generación automatizada de puertas (8)
        self.puertas = [Puerta(f"Puerta-{i+1}") for i in range(n_puertas)]
        #inventario -> DataFrame
        # Producto | id_producto | Cupo | Asignado | Total | Ubicado | Asignado | Parcialmente Asignado | Perdido
        self.inventario = pd.read_csv(path_inventario)

    def asignar_camiones_a_puertas(self, patio):
        """
        Asigna los camiones del patio a las puertas disponibles en el CEDIS.
        """
        for puerta in self.puertas:
            if puerta.estado == 'libre' and patio.camiones_en_espera:
                patio.asignar_camion_a_puerta(self)

In [57]:
import pandas as pd
import random
import string
from datetime import datetime, timedelta

# Generar datos sintéticos de camiones
def generar_camiones(n_camiones):
    ids_ordenes = [f"O-{i+1}" for i in range(n_camiones)]  # Generar IDs de ordenes únicos
    camiones = []
    for i in range(n_camiones):
        id_camion = f"C-{i+1}"
        # id_orden = ids_ordenes[i]
        
        num_prductos = random.choice([1,2,3,4])
        if num_prductos == 1:
            contenido = [{random.choice(['ProductoA', 'ProductoB', 'ProductoC', 'ProductoD']):random.choice([50,100,100])}]
        elif num_prductos == 2:
            productos = ['ProductoA', 'ProductoB', 'ProductoC', 'ProductoD']
            p1 = random.choice(productos)
            productos.remove(p1)
            p2 = random.choice(productos)
            contenido = [{p1:random.choice([50,100,100])},
                         {p2:random.choice([50,100,100])}]
        elif num_prductos == 3:
            productos = ['ProductoA', 'ProductoB', 'ProductoC', 'ProductoD']
            p1 = random.choice(productos)
            productos.remove(p1)
            p2 = random.choice(productos)
            productos.remove(p2)
            p3 = random.choice(productos)
            contenido = [{p1:random.choice([50,100,100])},
                         {p2:random.choice([50,100,100])},
                         {p3:random.choice([50,100,100])}]
        elif num_prductos == 4:
            productos = ['ProductoA', 'ProductoB', 'ProductoC', 'ProductoD']
            contenido = [{productos[0]:random.choice([50,100,100])},
                         {productos[1]:random.choice([50,100,100])},
                         {productos[2]:random.choice([50,100,100])},
                         {productos[3]:random.choice([50,100,100])}]


        # Generar una fecha de salida aleatoria dentro de los últimos 30 días
        fecha_salida = (datetime.today() - timedelta(days=random.randint(0, 30))).strftime("%m/%d/%Y")
        # Generar un origen aleatorio para cada camión
        origen = random.choice(["Planta A", "Planta B", "Sucursal X", "Centro Y"])
        camiones.append([id_camion, contenido, fecha_salida, origen])
    
    # Guardar en un DataFrame y exportar a CSV
    df_camiones = pd.DataFrame(camiones, columns=["id_camion", "contenido", "fecha_salida", "origen"])
    df_camiones.to_csv(r"D:\TEC\Universidad\Séptimo Semestre\IA AVANZADA II\Bimbo\camiones.csv", index=False)
    print("camiones.csv generado con éxito.")
    
# Generar datos sintéticos de órdenes
def generar_ordenes(n_ordenes):
    ordenes = []
    tipos_de_orden = ["fp", "pk"]  # Tipos de orden
    for i in range(n_ordenes):
        id_orden = f"O-{i+1}"
        # Generar un identificador de cliente aleatorio
        orden_cliente = ''.join(random.choices(string.ascii_uppercase + string.digits, k=5))
        # Seleccionar un tipo de orden aleatoriamente
        tipo_de_orden = random.choice(tipos_de_orden)
        
        # Generar productos con cantidades aleatorias
        productos = {}
        n_productos = random.randint(1, 5)  # Cantidad de productos por orden (entre 1 y 5)
        for _ in range(n_productos):
            producto_id = random.choice(['ProductoA', 'ProductoB', 'ProductoC', 'ProductoD'])
            cantidad_original = random.randint(1, 200)
            cantidad_solicitada = cantidad_original
            cantidad_asignada = random.randint(0, cantidad_solicitada)  # Asignado es entre 0 y lo solicitado
            cantidad_empaquetada = random.randint(0, cantidad_asignada)  # Empaquetado no supera lo asignado
            productos[producto_id] = [cantidad_original, cantidad_solicitada, cantidad_asignada, cantidad_empaquetada]
        
        # Convertir el diccionario a cadena de texto para guardarlo en el CSV
        productos_str = str(productos)
        ordenes.append([id_orden, orden_cliente, tipo_de_orden, productos_str])
    
    # Guardar en un DataFrame y exportar a CSV
    df_ordenes = pd.DataFrame(ordenes, columns=["id_orden", "orden_cliente", "tipo_de_orden", "productos"])
    df_ordenes.to_csv(r"D:\TEC\Universidad\Séptimo Semestre\IA AVANZADA II\Bimbo\ordenes.csv", index=False)
    print("ordenes.csv generado con éxito.")

# Generar datos sintéticos de inventario
def generar_inventario(n_productos):
    inventario = []
    productos = ['ProductoA', 'ProductoB', 'ProductoC', 'ProductoD']
    for i in range(n_productos):
        id_producto = f"P-{i+1}"
        # producto = random.choice(['ProductoA', 'ProductoB', 'ProductoC', 'ProductoD'])
        producto = productos[i]
        cupo = random.randint(50, 1000)  # Cupo total del producto en el inventario
        asignado = random.randint(0, cupo)  # Asignado no puede superar el cupo
        total = cupo + random.randint(0, 100)  # Total puede ser mayor que el cupo (productos en tránsito o no ubicados)
        ubicado = random.randint(0, total)  # Cantidad ubicada en almacén
        parcialmente_asignado = random.randint(0, total - asignado)  # Cantidad parcialmente asignada
        perdido = random.randint(0, 20)  # Productos perdidos
        inventario.append([producto, id_producto, cupo, asignado, total, ubicado, asignado, parcialmente_asignado, perdido])
    
    # Guardar en un DataFrame y exportar a CSV
    df_inventario = pd.DataFrame(inventario, columns=["Producto", "id_producto", "Cupo", "Asignado", "Total", "Ubicado", "Asignado", "Parcialmente Asignado", "Perdido"])
    df_inventario.to_csv(r"D:\TEC\Universidad\Séptimo Semestre\IA AVANZADA II\Bimbo\inventario.csv", index=False)
    print("inventario.csv generado con éxito.")

# Generar los datasets
generar_camiones(16)
generar_ordenes(50)
generar_inventario(4)

camiones.csv generado con éxito.
ordenes.csv generado con éxito.
inventario.csv generado con éxito.


In [58]:
import ast
# Función para cargar el CSV y crear los objetos Camion
def cargar_camiones(csv_file):
    # Carga el CSV en un DataFrame
    df = pd.read_csv(csv_file)
    
    # Convierte la columna 'contenido' a lista de diccionarios
    df['contenido'] = df['contenido'].apply(ast.literal_eval)

    # Crea una lista de instancias de Camion
    camiones = [
        Camion(row['id_camion'], row['contenido'], row['fecha_salida'], row['origen'])
        for _, row in df.iterrows()
    ]
    
    return camiones

# Llamada a la función con el archivo CSV
archivo_csv = 'data/camiones.csv' # Asegúrate de cambiarlo por la ruta a tu archivo CSV
lista_camiones = cargar_camiones(archivo_csv)

# Imprime la lista de camiones
for camion in lista_camiones:
    print(camion.id_camion,camion.contenido,'|',camion.origen)

C-1 [{'ProductoC': 100}, {'ProductoD': 50}, {'ProductoA': 100}] | Planta A
C-2 [{'ProductoD': 100}] | Sucursal X
C-3 [{'ProductoD': 100}] | Planta B
C-4 [{'ProductoA': 50}, {'ProductoB': 100}, {'ProductoC': 100}, {'ProductoD': 50}] | Planta B
C-5 [{'ProductoD': 100}] | Planta B
C-6 [{'ProductoD': 100}, {'ProductoB': 50}, {'ProductoC': 50}] | Sucursal X
C-7 [{'ProductoB': 100}, {'ProductoD': 100}, {'ProductoC': 50}] | Centro Y
C-8 [{'ProductoC': 100}, {'ProductoB': 100}, {'ProductoD': 50}] | Planta A
C-9 [{'ProductoA': 100}, {'ProductoC': 50}, {'ProductoD': 50}] | Centro Y
C-10 [{'ProductoA': 100}, {'ProductoB': 100}, {'ProductoC': 50}, {'ProductoD': 100}] | Planta A
C-11 [{'ProductoD': 100}, {'ProductoC': 100}, {'ProductoA': 100}] | Planta B
C-12 [{'ProductoB': 100}, {'ProductoC': 100}] | Centro Y
C-13 [{'ProductoC': 100}, {'ProductoB': 50}] | Planta A
C-14 [{'ProductoC': 100}, {'ProductoD': 100}] | Sucursal X
C-15 [{'ProductoA': 100}] | Planta A
C-16 [{'ProductoA': 50}] | Centro Y


In [59]:
# Función para cargar el CSV y crear los objetos Ordem
def cargar_ordenes(csv_files):
    # Carga el CSV en un DataFrame
    df = pd.read_csv(csv_files)

    # Convierte la columna 'productos' a lista de diccionarios
    df['productos'] = df['productos'].apply(ast.literal_eval)

    # Crea una lista de instancias de Orden
    ordenes = [
        Orden(row['id_orden'], row['orden_cliente'], row['tipo_de_orden'], row['productos'])
        for _, row in df.iterrows()
    ]

    return ordenes

# Llamada a la función con el archivo CSV
archivo_csv = 'data/ordenes.csv'  # Asegúrate de cambiarlo por la ruta a tu archivo CSV
lista_ordenes = cargar_ordenes(archivo_csv)

# Imprime la lista de ordenes
for orden in lista_ordenes:
    print(orden.id_orden, orden.orden_cliente, orden.tipo_de_orden, orden.productos)

O-1 98KV2 pk {'ProductoA': [33, 33, 23, 14], 'ProductoC': [96, 96, 0, 0], 'ProductoB': [62, 62, 15, 0]}
O-2 3307E pk {'ProductoC': [112, 112, 47, 31], 'ProductoB': [45, 45, 41, 27], 'ProductoA': [134, 134, 28, 24], 'ProductoD': [173, 173, 146, 37]}
O-3 R8D72 pk {'ProductoB': [181, 181, 152, 101], 'ProductoC': [141, 141, 20, 10], 'ProductoA': [195, 195, 151, 46]}
O-4 PZUC1 fp {'ProductoA': [150, 150, 115, 71], 'ProductoB': [7, 7, 6, 6]}
O-5 4T3R8 fp {'ProductoC': [148, 148, 134, 126], 'ProductoB': [183, 183, 1, 0], 'ProductoD': [69, 69, 20, 17]}
O-6 L2NS0 pk {'ProductoD': [190, 190, 52, 36], 'ProductoA': [58, 58, 11, 7], 'ProductoC': [106, 106, 44, 23]}
O-7 S8IYP fp {'ProductoD': [92, 92, 68, 51], 'ProductoA': [13, 13, 6, 0]}
O-8 DSIST pk {'ProductoA': [66, 66, 16, 0], 'ProductoC': [175, 175, 173, 71]}
O-9 05CAL pk {'ProductoC': [7, 7, 0, 0], 'ProductoA': [114, 114, 30, 0]}
O-10 POWGL fp {'ProductoC': [47, 47, 35, 19], 'ProductoD': [198, 198, 9, 6], 'ProductoB': [83, 83, 31, 21]}
O-11 W

In [60]:
# Función para cargar el CSV y crear el objeto CEDIS
def cargar_cedis(n_puertas, path_inventario):
    cedis = CEDIS(n_puertas, path_inventario)
    return cedis

# Llamada a la función con el número de puertas y el archivo CSV de inventario
n_puertas = 8  # Número de puertas en el CEDIS
archivo_inventario = 'data/inventario.csv'  # Asegúrate de cambiarlo por la ruta a tu archivo CSV
cedis = cargar_cedis(n_puertas, archivo_inventario)

# Imprime el inventario del CEDIS
cedis.inventario

,Producto,id_producto,Cupo,Asignado,Total,Ubicado,Asignado.1,Parcialmente Asignado,Perdido
0,ProductoA,P-1,269,197,295,78,197,90,10
1,ProductoB,P-2,981,705,1050,227,705,34,0
2,ProductoC,P-3,275,62,346,119,62,218,11
3,ProductoD,P-4,379,376,383,252,376,1,16


# Optimización

### Modelos:
**Modelo 1:** Maximizar la descarga de producto escaso en almacén.

**Modelo 2:** Minimizar el tiempo de descarga de los camiones.

### Suposiciones:

- Suponemos que las fosas tienen equitativamente el mismo uso.
- Suponemos que tienen 1 acceso.

### Resultado esperado:

Ambos modelos entregarán como resultado una lista del órden en la que los camiones deberán ser atendidos dependiendo del objetivo que se tenga.

---
# Modelo para Maximizar la descarga del producto escaso

## Variables y Parámetros:

### Variables de Decisión

- 

### Parámetros


- $P_i \geq 1$: Número de pallets que transporta el camión $i$. Se obtiene de la documentación del camión.

- $u_{ij} \geq 0$: Unidades del producto $j$ que transporta el camión $i$. Se obtiene del manifiesto de carga del camión.

- $\gamma_j > 0$: Importancia del producto $j$. Se calcula utilizando información histórica y análisis de datos.

- $D_j \geq 0$: Demanda mínima requerida del producto $j$. Se establece según las necesidades de inventario y pedidos pendientes.

- $B \geq 1$: Número de bahías de descarga disponibles. Es un dato conocido de la infraestructura.

- $\gamma_i > 0$: Prioridad del camión $i$, calculada en función de los productos que transporta.

- $\gamma_o > 0$: Prioridad de la orden $o$, calculada en función del tipo de orden y la cantidad de productos pendientes. 

## Cálculo de las Variables Clave

### Cálculo de la Importancia de los Productos ($\gamma_j$)

La importancia de los productos $\gamma_j$ se calcula utilizando información histórica y análisis de datos. Los factores considerados pueden incluir:

1. Demanda histórica ($H_j$): Volumen de ventas o demanda del producto $j$ en periodos anteriores.
2. Disponibilidad en el inventario ($I_j$): Cantidad de producto $j$ disponible en el inventario.
3. Cantidad solicitada ($Q_j$): Cantidad de producto $j$ solicitada en la orden.
4. Penalización por falta ($F_j$): Costo asociado a no satisfacer la demanda del producto $j$.

Para combinar estos factores, se normalizan los valores y se asignan pesos ($w_k$):

$$
\gamma_j = w_1 \times \text{Norm}(H_j) + w_2 \times \text{Norm}(I_j) + w_3 \times \text{Norm}(Q_j) + w_4 \times \text{Norm}(F_j)
$$

Donde:

- $\text{Norm}(X)$ es la normalización de $X$ al rango [0,1].
- $w_1 + w_2 + w_3 + w_4 = 1$.

### Cálculo de la Prioridad del Camión ($\gamma_i$)

La prioridad del camión $i$ se calcula sumando las importancias de los productos que transporta, ponderadas por las cantidades:

$$
\gamma_i = \sum_{j} (\gamma_j \times u_{ij})
$$

Cómo se obtiene:

Se multiplican las unidades de cada producto en el camión por la importancia del producto y se suman para obtener la prioridad total del camión.

### Cálculo de la Prioridad de la Orden ($\gamma_o$)

1. Cantidad solicitada de productos ($Q_o$): Volumen de productos $j$ solicitados en la orden.
2. Productos pendientes de entrega ($P_o$): Cantidad de productos $j$ de la orden que aún no han sido entregados.
3. Tipo de orden ($T_o$): Valor binario donde $T_o = 1$ si la orden es de tipo "fk" (mayor prioridad), y $T_o = 0$ si es de tipo "pk".

Para combinar estos factores, se normalizan los valores y se asignan pesos ($w_k$):

$$
\gamma_o = w_1 \times \text{Norm}(Q_o) + w_2 \times \text{Norm}(P_o) + w_3 \times T_o
$$

## Función Objetivo

Maximizar el cumplimiento de ordenes faltantes:

$$
\text{Maximizar} \ Z = \sum_{i} \gamma_i \times \gamma_o
$$

## Restricciones

1. No solapamiento en las bahías:
   $$
   t_i \geq t_{\text{previo}} + S_{\text{previo}}, \ \forall i \ \text{en la misma bahía}
   $$

2. Cumplimiento de demandas mínimas:
   $$
   \sum_{i} u_{ij} \geq D_j, \ \forall j
   $$


---
# Modelo para Minimizar el tiempo de descarga
## Variables y Parámetros:

### Variables de Decisión
- $ti \geq 0$: Tiempo de inicio de la descarga del camión $i$ (en minutos). Se calcula según la asignación y secuencia en las fosas.

- $S_i > 0$: Tiempo de servicio o descarga del camión $i$ (en minutos). Se obtiene multiplicando el número de pallets por el tiempo de descarga por pallet.

- $W_i \geq 0$: Tiempo de espera del camión $i$ antes de iniciar su descarga. Se calcula como $W_i = t_i - A_i$.

### Parámetros

- $A_i \geq 0$: Tiempo de llegada del camión $i$ (en minutos desde un origen común). Se obtiene de la programación de llegadas o registros de entrada.

- $P_i \geq 1$: Número de pallets que transporta el camión $i$. Se obtiene de la documentación del camión.

- $T_p > 0$: Tiempo fijo para descargar un pallet (en minutos). Es un valor constante determinado por la operación (por ejemplo, $T_p = 5$ minutos).

- $B \geq 1$: Número de fosas de descarga disponibles. Es un dato conocido de la infraestructura.

## Cálculo de las Variables Clave

### Tiempo de Servicio ($S_i$)

El tiempo de servicio del camión $i$ es:

$$
S_i = P_i \times T_p
$$

Cómo se obtiene:

Se multiplica el número de pallets $P_i$ por el tiempo de descarga por pallet $T_p$.

### Cálculo de Tiempos de Inicio ($t_i$)

El tiempo de inicio de descarga del camión $i$ depende de:

- El tiempo de llegada $A_i$.
- La disponibilidad de la bahía asignada.
- La secuencia de camiones en la bahía.

Para el camión $i$ en la posición $n$ en la bahía $k$:

$$
t_i = \max(A_i, t_{\text{previo}} + S_{\text{previo}})
$$

Donde $t_{\text{previo}} + S_{\text{previo}}$ es el tiempo en que la bahía estará disponible tras el camión anterior.

## Función Objetivo

Minimizar el tiempo total ponderado de espera y servicio:

$$
\text{Minimizar} \ Z = \sum_{i} t_i + S_i
$$

## Restricciones

1. No solapamiento en las fosas:
   $$
   t_i \geq t_{\text{previo}} + S_{\text{previo}}, \ \forall i \ \text{en la misma bahía}
   $$

2. Tiempo de inicio no antes de la llegada:
   $$
   t_i \geq A_i, \ \forall i
   $$